In [15]:
import numpy as np
import cv2
import os
import trimesh
import json

def get_k_w2c_flame(datadir, cam_id):
    with open(datadir, 'r') as file:
        params = json.load(file)
    
    extrinsic = np.array(params['world_2_cam'][cam_id])
    # 提取旋转矩阵 R
    R = extrinsic[:3, :3]
    # 提取平移向量 T
    T = extrinsic[:3, 3]
    # 提取相机内参
    K = np.array(params['intrinsics'])
    
    return K, R, T

def project_mesh_overlay(obj_path, K, R, t, image_path, output_path, alpha=0.5, color=(0, 0, 255)):
    """
    将3D网格投影到相机视角并叠加到原图
    
    参数：
    - obj_path:   .obj网格文件路径
    - K:          相机内参矩阵 (3x3 numpy数组)
    - R:          相机旋转矩阵 (3x3 numpy数组)
    - t:          相机平移向量 (3x1 numpy数组)
    - image_path: 原始图像路径
    - output_path: 输出图像保存路径
    - alpha:      透明度 (0.0-1.0)
    - color:      网格显示颜色 (BGR元组)
    """
    # 读取原始图像
    image = cv2.imread(image_path)
    if image is None:
        raise ValueError("无法读取原图")
    height, width = image.shape[:2]
    
    # 创建透明覆盖层
    overlay = image.copy()
    
    # 加载3D网格
    mesh = trimesh.load(obj_path)
    vertices = mesh.vertices
    faces = mesh.faces

    # 坐标系变换: 世界坐标 -> 相机坐标
    cam_coords = (R @ vertices.T + t.reshape(3, 1)).T
    
    # 投影到图像平面
    proj_points = []
    for X, Y, Z in cam_coords:
        if Z <= 0:  # 剔除相机背后的点
            proj_points.append((np.nan, np.nan))
            continue
        
        # 透视投影
        u = (K[0, 0] * X / Z) + K[0, 2]
        v = (K[1, 1] * Y / Z) + K[1, 2]
        
        # 边界检查
        if 0 <= u < width and 0 <= v < height:
            proj_points.append((u, v))
        else:
            proj_points.append((np.nan, np.nan))
    
    proj_points = np.array(proj_points)

    # 光栅化每个三角形
    for face in faces:
        # 获取三个顶点的投影坐标
        tri_points = proj_points[face]
        
        # 跳过无效三角形
        if np.isnan(tri_points).any():
            continue
        
        # 转换为整数坐标
        pts = tri_points.reshape(-1, 1, 2).astype(np.int32)
        
        # 绘制填充三角形
        cv2.fillPoly(overlay, [pts], color)

    # 透明度混合
    result = cv2.addWeighted(image, 1 - alpha, overlay, alpha, 0)
    
    # 保存结果
    cv2.imwrite(output_path, result)

def create_collage(image_folder, output_path, rows=2, cols=8, target_width=200):
    """
    创建图像拼贴画
    
    参数：
    - image_folder: 包含输入图像的文件夹路径
    - output_path: 拼接结果保存路径
    - rows: 行数
    - cols: 列数
    - target_width: 每张小图的统一宽度（高度按比例自动计算）
    """
    # 获取排序后的图像文件列表
    image_files = sorted([f for f in os.listdir(image_folder) if f.endswith('.jpg')])  
    
    # 读取并调整所有图像尺寸
    images = []
    for f in image_files[:rows*cols]:  # 只取前16张
        img = cv2.imread(os.path.join(image_folder, f))
        if img is not None:
            # 计算缩放比例
            h, w = img.shape[:2]
            scale = target_width / w
            target_size = (target_width, int(h * scale))
            resized = cv2.resize(img, target_size, interpolation=cv2.INTER_AREA)
            images.append(resized)
    
    # 创建空白画布
    canvas_width = cols * target_width
    canvas_height = sum([img.shape[0] for img in images[:rows*cols:cols]]) // rows  # 取每行第一张的高度
    canvas = np.zeros((canvas_height*rows, canvas_width, 3), dtype=np.uint8)
    
    # 拼接图像
    for i in range(rows):
        for j in range(cols):
            idx = i*cols + j
            if idx >= len(images):
                break
            img = images[idx]
            # 计算粘贴位置
            y_start = i * canvas_height
            y_end = y_start + img.shape[0]
            x_start = j * target_width
            x_end = x_start + img.shape[1]
            # 居中放置
            y_pad = (canvas_height - img.shape[0]) // 2
            canvas[y_start+y_pad:y_end+y_pad, x_start:x_end] = img
    
    # 保存结果
    cv2.imwrite(output_path, canvas)

# 使用示例 --------------------------------------------------
if __name__ == "__main__":
    # 示例参数（需要替换为实际参数）
    id = "139"
    images_pth = f'/media/VHAP/data/{id}/EMO-1-shout+laugh/images'
    image_list = os.listdir(images_pth)
    obj_path = f"/media/VHAP/data/{id}/output/2025-04-01_13-23-56/eval_30/mesh/frame_00000.obj"  
    camera_pth = f'/media/VHAP/data/camera_params/{id}/camera_params.json'
    save_pth = f'./out_mesh_fig/{id}'
    os.makedirs(save_pth,exist_ok=True)
    for image in image_list:
        image_path = os.path.join(images_pth, image)
        
        output_path =os.path.join(save_pth, f'{image}.jpg')        # 输出路径
        K, R, t = get_k_w2c_flame(camera_pth, image.split('_')[1])
        project_mesh_overlay(
            obj_path=obj_path,
            K=K,
            R=R,
            t=t,
            image_path=image_path,
            output_path=output_path,
            alpha=0.4,
            color=(256, 0, 0)  # 红色
        )
    # 使用示例
    create_collage(
        image_folder=f"./out_mesh_fig/{id}",
        output_path=os.path.join(save_pth, "collage_result.jpg"),
        rows=2,
        cols=8,
        target_width=200  # 根据需求调整
    )

